In [ ]:
import sys
import os


sys.path.append(os.path.abspath("../..")) #make the folder "automl" part of this

RESULTS_PATH = 'results.csv'
OPTUNA_DATABASE = 'study_results.db'


In [ ]:
from automl.loggers.result_logger import ResultLogger
import optuna
import optuna.visualization as vis
from automl.utils.optuna_utils import load_study_from_database
import matplotlib.pyplot as plt


In [ ]:
AGGREGATE_NUMBER = 20 #the number of neighbor points to sum to plot the needed graphs

In [ ]:
class Meta(type):
    
    def __prepare__(cls, name, **kwargs):
        print("Meta __prepare__")
        return {"attribute_name": "value"}
        
    def __new__(cls, name, bases, attrs):
        print("Meta __new__")
        return super().__new__(cls, name, bases, attrs)
    
    def __init__(cls, name, bases, attrs):
        print("Meta __init__")
        super().__init__(name, bases, attrs)
        
class MyClass(metaclass=Meta):
    
    print("Class body")
    
    attribute_name_2 = "value_2"
    
    def __init__(self):
        print("MyClass __init__")
    
print(MyClass.attribute_name)
print(MyClass.attribute_name_2)


# Load the experiment

In [ ]:
#experiment_path = 'data\\experiments\\HyperparameterOptimizationPipeline_3'
experiment_path = 'archived_data\\first'

# Evaluation of HyperparameterOptimizationPipeline

In [ ]:
hyperparameter_optimization_results : ResultLogger = ResultLogger(input={
                                        "logger_directory" : experiment_path,
                                        "filename" : RESULTS_PATH
                                      })

hyperparameter_optimization_results.proccess_input()

In [ ]:
optuna_study = load_study_from_database(database_path=hyperparameter_optimization_results.lg.logDir + '\\' + OPTUNA_DATABASE)


## Hyperparameter Study

In [ ]:
fig = vis.plot_param_importances(optuna_study)
fig.show()

In [ ]:


fig = vis.plot_parallel_coordinate(optuna_study)
fig.show()

In [ ]:
fig = vis.plot_intermediate_values(optuna_study)
fig.show()

In [ ]:
fig = vis.plot_optimization_history(optuna_study)
fig.show()

In [ ]:
parameters_to_plot = ["hidden_size", "hidden_layers"]

fig = vis.plot_contour(optuna_study, params=parameters_to_plot)
fig.show()

In [ ]:
parameters_to_plot = ["epsilon_start", "epsilon_decay", "epsilon_end"]

fig = vis.plot_contour(optuna_study, params=parameters_to_plot)
fig.show()

# Global evaluation of configurations

In [ ]:

results_of_configurations : dict[str, ResultLogger] = {}

for configuration_name in os.listdir(experiment_path):
    configuration_path = os.path.join(experiment_path, configuration_name)
    
    if os.path.isdir(configuration_path):  # Ensure it's a file, not a subdirectory
        
        results_of_configurations[configuration_name] = ResultLogger(input={
                                        "logger_directory" : configuration_path,
                                        "filename" : RESULTS_PATH
                                      })
        results_of_configurations[configuration_name].proccess_input()


In [ ]:
print(f"Configurations:  {results_of_configurations.keys()}")

# Global view of performance

In [ ]:
hyperparameter_optimization_results.plot_bar_graph(x_axis='experiment', y_axis='result')

## Pruned Performances

In [ ]:
pruned_optuna_trials = [trial for trial in optuna_study.trials if trial.state == optuna.trial.TrialState.PRUNED]

pruned_optuna_trials_per_steps : dict[int, list[optuna.trial.FrozenTrial]] = {} #the pruned trials by the number of completed steps

for pruned_optuna_trial in pruned_optuna_trials:
    
    n_completed_steps = len(pruned_optuna_trial.intermediate_values)
    
    try:
        list_of_pruned = pruned_optuna_trials_per_steps[n_completed_steps]
    
    except:
        list_of_pruned = []
        pruned_optuna_trials_per_steps[n_completed_steps] = list_of_pruned    
        
    list_of_pruned.append(pruned_optuna_trial)
    
    

pruned_trials = [f'configuration_{trial.number + 1}' for trial in optuna_study.trials if trial.state == optuna.trial.TrialState.PRUNED]

print(f"Pruned trials: {pruned_trials}")


In [ ]:
for n_completed_steps in pruned_optuna_trials_per_steps.keys():
    
    pruned_optuna_trials = pruned_optuna_trials_per_steps[n_completed_steps] 
    
    pruned_trials = [f'configuration_{trial.number + 1}' for trial in pruned_optuna_trials ]

    for configuration_name in pruned_trials:
        
        results_logger = results_of_configurations[configuration_name]
    
        #results_logger.plot_graph(x_axis='episode', y_axis=[('total_reward', name)], to_show=False)
        results_logger.plot_confidence_interval(x_axis='episode', y_column='total_reward',show_std=False, to_show=False, y_values_label=configuration_name, aggregate_number=AGGREGATE_NUMBER)
        #results_logger.plot_linear_regression(x_axis='episode', y_axis='total_reward', to_show=False, y_label=configuration_name + '_linear')
        
    plt.show()

## Completed Trials Evaluation

In [ ]:
completed_optuna_trials = [trial for trial in optuna_study.trials if trial.state == optuna.trial.TrialState.COMPLETE]

completed_optuna_trials.sort(key=lambda trial: trial.value) # sort given the trial value

### Worst Completed Performances

In [ ]:
N_WORST = 3

In [ ]:

worst_optuna_trials = completed_optuna_trials[0:N_WORST]

worst_configurations_to_study = [f"configuration_{trial.number + 1}" for trial in worst_optuna_trials]

print(f"Worst configurations to study: {worst_configurations_to_study}")

In [ ]:
for configuration_name in worst_configurations_to_study:
    
    results_logger = results_of_configurations[configuration_name]

    #results_logger.plot_graph(x_axis='episode', y_axis=[('total_reward', name)], to_show=False)
    results_logger.plot_confidence_interval(x_axis='episode', y_column='total_reward',show_std=False, to_show=False, y_values_label=configuration_name, aggregate_number=AGGREGATE_NUMBER)
    #results_logger.plot_linear_regression(x_axis='episode', y_axis='total_reward', to_show=False, y_label=configuration_name + '_linear')
    
plt.show()

## Best Performances

In [ ]:
N_BEST = 3

In [ ]:
best_optuna_trials = completed_optuna_trials[(len(completed_optuna_trials) - 1 - N_BEST):]

best_configurations_to_study = [f"configuration_{trial.number + 1}" for trial in best_optuna_trials]

print(f"Best configurations to study: {best_configurations_to_study}")

In [ ]:
for configuration_name in best_configurations_to_study:
    
    results_logger = results_of_configurations[configuration_name]

    #results_logger.plot_graph(x_axis='episode', y_axis=[('total_reward', name)], to_show=False)
    results_logger.plot_confidence_interval(x_axis='episode', y_column='total_reward',show_std=False, to_show=False, y_values_label=configuration_name, aggregate_number=AGGREGATE_NUMBER)
    #results_logger.plot_linear_regression(x_axis='episode', y_axis='total_reward', to_show=False, y_label=configuration_name + '_linear')
    
plt.show()

# Smaller study

In [ ]:
configurations_to_study = ['configuration_45']

In [ ]:


for configuration_name in configurations_to_study:
    
    results_logger = results_of_configurations[configuration_name]

    #results_logger.plot_graph(x_axis='episode', y_axis=[('total_reward', name)], to_show=False)
    results_logger.plot_confidence_interval(x_axis='episode', y_column='total_reward',show_std=True, to_show=False, y_values_label=configuration_name, aggregate_number=AGGREGATE_NUMBER)
    results_logger.plot_linear_regression(x_axis='episode', y_axis='total_reward', to_show=False, y_label=configuration_name + '_linear')
    
plt.show()

In [ ]:
agents_to_study : dict[str, ResultLogger]= {}

for configuration_name in configurations_to_study:
    
    results_logger = results_of_configurations[configuration_name]
    
    for agent_name in ["agent_1", "agent_2"]:
      
        agent_results_logger = ResultLogger(input={
                                            "logger_directory" : f"{results_logger.lg.logDir}\\{agent_name}",
                                            "filename" : RESULTS_PATH
                                          })

        agents_to_study[f"{configuration_name}_{agent_name}"] = agent_results_logger
        
        agent_results_logger.proccess_input()


In [ ]:
for agent_name, agent_results_logger in agents_to_study.items():
    
    #results_logger.plot_graph(x_axis='episode', y_axis=[('total_reward', name)], to_show=False)
    agent_results_logger.plot_confidence_interval(x_axis='episode', y_column='total_reward',show_std=False, to_show=False, y_values_label=agent_name, aggregate_number=AGGREGATE_NUMBER)
    
plt.show()